# Prediction Model

We use an Elo rating system updated sequentially using game outcomes and margin of victory.
For upcoming games, we adjust the base Elo difference using rolling efficiency metrics—offensive yards per play, defensive yards allowed per play, and turnover margin—computed from recent games only.
This produces a pre-game win probability without training a statistical or machine-learning model.

In [1]:
import numpy as np
import pandas as pd
import nfl_data_py as nfl

In [2]:
SEASON = 2025

sched = nfl.import_schedules([SEASON]).copy()

# Inspect once if you want:
# print(sched.columns)

# Robust column picking (schedule schema varies)
def pick(df, options):
    for c in options:
        if c in df.columns:
            return c
    raise KeyError(f"None of these columns found: {options}")

home_team_col = pick(sched, ["home_team", "home_team_abbr"])
away_team_col = pick(sched, ["away_team", "away_team_abbr"])
home_score_col = pick(sched, ["home_score", "score_home"])
away_score_col = pick(sched, ["away_score", "score_away"])
week_col       = pick(sched, ["week", "game_week"])
game_id_col    = pick(sched, ["game_id", "gameid", "gsis_id"])

sched = sched.rename(columns={
    home_team_col: "home_team",
    away_team_col: "away_team",
    home_score_col: "home_score",
    away_score_col: "away_score",
    week_col: "week",
    game_id_col: "game_id",
})

# If there's a game type column, keep regular season
if "game_type" in sched.columns:
    sched = sched[sched["game_type"].astype(str).str.upper().str.startswith("REG")].copy()
elif "season_type" in sched.columns:
    sched = sched[sched["season_type"].astype(str).str.upper().str.startswith("REG")].copy()

sched["week_num"] = pd.to_numeric(sched["week"], errors="coerce")

played = sched[sched["home_score"].notna() & sched["away_score"].notna()].copy()
future = sched[sched["home_score"].isna() | sched["away_score"].isna()].copy()

print("Played games:", len(played), "Future games:", len(future))


Played games: 225 Future games: 47


In [3]:
pbp = nfl.import_pbp_data([SEASON]).copy()

# Regular season pbp only if present
if "season_type" in pbp.columns:
    pbp = pbp[pbp["season_type"] == "REG"].copy()
    
# Ensure game_id column
pbp_gid = pick(pbp, ["game_id", "gameid", "gsis_id"])
pbp = pbp.rename(columns={pbp_gid: "game_id"})

# Keep rush/pass plays
off = pbp[(pbp.get("pass", 0) == 1) | (pbp.get("rush", 0) == 1)].copy()

# Turnover flags
if "interception" not in off.columns: off["interception"] = 0
if "fumble_lost" not in off.columns:  off["fumble_lost"] = 0
off["giveaway"] = ((off["interception"] == 1) | (off["fumble_lost"] == 1)).astype(int)

# Offensive per game/team
off_agg = (
    off.groupby(["game_id", "posteam"])
      .agg(plays=("yards_gained", "size"),
           off_yards=("yards_gained", "sum"),
           giveaways=("giveaway", "sum"))
      .reset_index()
      .rename(columns={"posteam":"team"})
)
off_agg["off_ypp"] = off_agg["off_yards"] / off_agg["plays"]

# Defensive YPP allowed
def_agg = (
    off.groupby(["game_id", "defteam"])
      .agg(def_plays=("yards_gained","size"),
           def_yards_allowed=("yards_gained","sum"),
           takeaways=("giveaway","sum"))
      .reset_index()
      .rename(columns={"defteam":"team"})
)
def_agg["def_ypp"] = def_agg["def_yards_allowed"] / def_agg["def_plays"]

feats = off_agg.merge(def_agg[["game_id","team","def_ypp","takeaways"]], on=["game_id","team"], how="left")
feats["takeaways"] = feats["takeaways"].fillna(0)
feats["to_margin"] = feats["takeaways"] - feats["giveaways"]

feats = feats[["game_id","team","off_ypp","def_ypp","to_margin"]].copy()


2025 done.
Downcasting floats.


In [4]:
WINDOW = 6

# Long format: one row per team per game with week order from schedule
long_home = played[["game_id","week_num","home_team"]].rename(columns={"home_team":"team"})
long_away = played[["game_id","week_num","away_team"]].rename(columns={"away_team":"team"})
long = pd.concat([long_home, long_away], ignore_index=True)

long = long.merge(feats, on=["game_id","team"], how="left")
long = long.sort_values(["week_num","game_id","team"]).copy()

for col in ["off_ypp","def_ypp","to_margin"]:
    long[f"{col}_roll"] = (
        long.groupby("team")[col]
            .apply(lambda s: s.shift(1).rolling(WINDOW, min_periods=1).mean())
            .reset_index(level=0, drop=True)
    )

# Latest available rolling stats per team (what we use for future games)
latest = (
    long.sort_values(["week_num","game_id"])
        .groupby("team")
        .tail(1)
        .set_index("team")[["off_ypp_roll","def_ypp_roll","to_margin_roll"]]
        .rename(columns={"off_ypp_roll":"off_ypp","def_ypp_roll":"def_ypp","to_margin_roll":"to_margin"})
)
latest.head()


/var/folders/mk/354ykzvd4xb9dv44bwrmk9w80000gn/T/ipykernel_3836/3738686947.py:14: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda s: s.shift(1).rolling(WINDOW, min_periods=1).mean())
/var/folders/mk/354ykzvd4xb9dv44bwrmk9w80000gn/T/ipykernel_3836/3738686947.py:14: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .group

,off_ypp,def_ypp,to_margin
team,,,
ARI,3.939165,5.020949,0.333333
HOU,5.092477,5.046143,1.200000
ATL,6.090068,4.200350,0.333333
TB,5.021256,4.731764,-0.200000
BAL,4.562514,4.234678,0.500000


In [5]:
def win_prob_from_elo(elo_home, elo_away):
    return 1 / (1 + 10 ** ((elo_away - elo_home) / 400))

def eff_adj(home_team, away_team, latest_df, w_off=120, w_def=120, w_to=35):
    # If a team has no history yet, treat as league average (0 adjustment)
    if home_team not in latest_df.index or away_team not in latest_df.index:
        return 0.0

    h = latest_df.loc[home_team]
    a = latest_df.loc[away_team]

    d_off = h["off_ypp"] - a["off_ypp"]
    d_def = a["def_ypp"] - h["def_ypp"]   # lower def_ypp is better
    d_to  = h["to_margin"] - a["to_margin"]

    return w_off*d_off + w_def*d_def + w_to*d_to

def train_elo(played_sched, latest_df, k=20, hfa=55):
    teams = pd.unique(played_sched[["home_team","away_team"]].values.ravel("K"))
    elo = {t: 1500.0 for t in teams}

    for _, g in played_sched.sort_values(["week_num","game_id"]).iterrows():
        home, away = g["home_team"], g["away_team"]
        elo_home, elo_away = elo[home], elo[away]

        # Use rolling stats as-of that point is more correct, but for "this weekend" usage
        # this simpler version is fine: Elo learns from results; efficiency only shifts predictions.
        p = win_prob_from_elo(elo_home + hfa, elo_away)

        if g["home_score"] == g["away_score"]:
            s = 0.5
        else:
            s = 1.0 if g["home_score"] > g["away_score"] else 0.0

        mov = abs(g["home_score"] - g["away_score"])
        mov_mult = np.log(max(mov, 1) + 1)

        elo[home] = elo_home + k * mov_mult * (s - p)
        elo[away] = elo_away + k * mov_mult * ((1 - s) - (1 - p))

    return elo

elo = train_elo(played, latest, k=20, hfa=55)

def predict_future_games(future_sched, elo, latest_df, hfa=55):
    preds = []
    for _, g in future_sched.sort_values(["week_num","game_id"]).iterrows():
        game_id = g["game_id"]
        home, away = g["home_team"], g["away_team"]

        elo_home = elo.get(home, 1500.0)
        elo_away = elo.get(away, 1500.0)

        adj = eff_adj(home, away, latest_df)  # efficiency adjustment in Elo points
        p_home = win_prob_from_elo(elo_home + hfa + adj, elo_away)

        preds.append({
            "game_id": game_id,                 # ✅ include this
            "week": g.get("week", None),
            "home_team": home,
            "away_team": away,
            "eff_adj_elo_pts": adj,
            "home_win_prob": p_home,
        })
    return pd.DataFrame(preds)


preds = predict_future_games(future, elo, latest, hfa=55)
preds.head(20)


,game_id,week,home_team,away_team,eff_adj_elo_pts,home_win_prob
0,2025_16_ATL_ARI,16,ARI,ATL,-356.580293,0.126663
1,2025_16_BUF_CLE,16,CLE,BUF,-70.676064,0.189162
2,2025_16_CIN_MIA,16,MIA,CIN,450.292348,0.969248
3,2025_16_GB_CHI,16,CHI,GB,-138.408612,0.334528
4,2025_16_JAX_DEN,16,DEN,JAX,-124.081484,0.480867
5,2025_16_KC_TEN,16,TEN,KC,-230.357829,0.106150
6,2025_16_LAC_DAL,16,DAL,LAC,-123.129964,0.294266
7,2025_16_LV_HOU,16,HOU,LV,63.322861,0.912518
8,2025_16_MIN_NYG,16,NYG,MIN,220.679750,0.697276
9,2025_16_NE_BAL,16,BAL,NE,105.740004,0.571098


In [6]:
# See what date columns exist
[c for c in sched.columns if "date" in c.lower() or "game" in c.lower() or "time" in c.lower()]


['game_id',
 'game_type',
 'gameday',
 'gametime',
 'overtime',
 'old_game_id',
 'div_game']

In [7]:
# Make sure these exist in your schedule df:
# 'gameday' (YYYY-MM-DD), 'gametime' (like '1:00 PM' or '20:15')
# If gametime is missing for some games, set a default so parsing works.
sched = sched.copy()
sched["gametime"] = sched["gametime"].fillna("12:00 PM")

# Combine into datetime (naive first)
sched["kickoff_naive"] = pd.to_datetime(
    sched["gameday"].astype(str) + " " + sched["gametime"].astype(str),
    errors="coerce"
)

# NFL times are typically listed in US/Eastern in many datasets.
# Localize to US/Eastern then convert to Mexico City.
sched["kickoff_mx"] = (
    sched["kickoff_naive"]
    .dt.tz_localize("America/New_York", ambiguous="NaT", nonexistent="NaT")
    .dt.tz_convert("America/Mexico_City")
)

sched[["gameday","gametime","kickoff_mx"]].head()


,gameday,gametime,kickoff_mx
6991,2025-09-04,20:20,2025-09-04 18:20:00-06:00
6992,2025-09-05,20:00,2025-09-05 18:00:00-06:00
6993,2025-09-07,13:00,2025-09-07 11:00:00-06:00
6994,2025-09-07,13:00,2025-09-07 11:00:00-06:00
6995,2025-09-07,13:00,2025-09-07 11:00:00-06:00


In [8]:
start = pd.Timestamp("2025-12-18", tz="America/Mexico_City")
end   = pd.Timestamp("2025-12-23", tz="America/Mexico_City")  # exclusive

weekend = sched[(sched["kickoff_mx"] >= start) & (sched["kickoff_mx"] < end)].copy()

weekend[["week","home_team","away_team","kickoff_mx"]].sort_values("kickoff_mx")


,week,home_team,away_team,kickoff_mx
7215,16,SEA,LA,2025-12-18 19:15:00-06:00
7216,16,WAS,PHI,2025-12-20 16:00:00-06:00
7217,16,CHI,GB,2025-12-20 19:20:00-06:00
7218,16,CAR,TB,2025-12-21 12:00:00-06:00
7219,16,CLE,BUF,2025-12-21 12:00:00-06:00
7220,16,DAL,LAC,2025-12-21 12:00:00-06:00
7221,16,MIA,CIN,2025-12-21 12:00:00-06:00
7222,16,NO,NYJ,2025-12-21 12:00:00-06:00
7223,16,NYG,MIN,2025-12-21 12:00:00-06:00
7224,16,TEN,KC,2025-12-21 12:00:00-06:00


In [9]:
weekend_played = weekend[weekend["home_score"].notna() & weekend["away_score"].notna()].copy()
weekend_future = weekend[weekend["home_score"].isna() | weekend["away_score"].isna()].copy()

print("Weekend played:", len(weekend_played), "Weekend future:", len(weekend_future))


Weekend played: 1 Weekend future: 15


In [10]:
weekend_future.columns


Index(['game_id', 'season', 'game_type', 'week', 'gameday', 'weekday',
       'gametime', 'away_team', 'away_score', 'home_team', 'home_score',
       'location', 'result', 'total', 'overtime', 'old_game_id', 'gsis',
       'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_rest', 'home_rest',
       'away_moneyline', 'home_moneyline', 'spread_line', 'away_spread_odds',
       'home_spread_odds', 'total_line', 'under_odds', 'over_odds', 'div_game',
       'roof', 'surface', 'temp', 'wind', 'away_qb_id', 'home_qb_id',
       'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee',
       'stadium_id', 'stadium', 'week_num', 'kickoff_naive', 'kickoff_mx'],
      dtype='object')

In [11]:
preds_weekend = predict_future_games(weekend_future, elo, latest, hfa=55)
# Add kickoff time for readability
preds_weekend = preds_weekend.merge(
    weekend_future[["game_id","kickoff_mx"]],
    on="game_id",
    how="left"
).sort_values("kickoff_mx")

preds_weekend[["kickoff_mx","home_team","away_team","home_win_prob","eff_adj_elo_pts"]]


,kickoff_mx,home_team,away_team,home_win_prob,eff_adj_elo_pts
11,2025-12-20 16:00:00-06:00,WAS,PHI,0.281416,-79.823625
3,2025-12-20 19:20:00-06:00,CHI,GB,0.334528,-138.408612
1,2025-12-21 12:00:00-06:00,CLE,BUF,0.189162,-70.676064
2,2025-12-21 12:00:00-06:00,MIA,CIN,0.969248,450.292348
5,2025-12-21 12:00:00-06:00,TEN,KC,0.106150,-230.357829
6,2025-12-21 12:00:00-06:00,DAL,LAC,0.294266,-123.129964
8,2025-12-21 12:00:00-06:00,NYG,MIN,0.697276,220.679750
10,2025-12-21 12:00:00-06:00,NO,NYJ,0.571435,-58.848512
14,2025-12-21 12:00:00-06:00,CAR,TB,0.482561,-56.283575
0,2025-12-21 15:05:00-06:00,ARI,ATL,0.126663,-356.580293


In [12]:
preds_weekend = preds_weekend.copy()

preds_weekend["favorite"] = np.where(
    preds_weekend["home_win_prob"] >= 0.5,
    preds_weekend["home_team"],
    preds_weekend["away_team"]
)

preds_weekend["fav_win_prob"] = np.where(
    preds_weekend["home_win_prob"] >= 0.5,
    preds_weekend["home_win_prob"],
    1 - preds_weekend["home_win_prob"]
)

def confidence(p):
    if p < 0.55:
        return "Coin flip"
    elif p < 0.65:
        return "Lean"
    elif p < 0.75:
        return "Strong"
    else:
        return "Very strong"

preds_weekend["confidence"] = preds_weekend["fav_win_prob"].apply(confidence)

preds_weekend[[
    "kickoff_mx",
    "favorite",
    "home_team",
    "away_team",
    "fav_win_prob",
    "confidence",
]]


,kickoff_mx,favorite,home_team,away_team,fav_win_prob,confidence
11,2025-12-20 16:00:00-06:00,PHI,WAS,PHI,0.718584,Strong
3,2025-12-20 19:20:00-06:00,GB,CHI,GB,0.665472,Strong
1,2025-12-21 12:00:00-06:00,BUF,CLE,BUF,0.810838,Very strong
2,2025-12-21 12:00:00-06:00,MIA,MIA,CIN,0.969248,Very strong
5,2025-12-21 12:00:00-06:00,KC,TEN,KC,0.893850,Very strong
6,2025-12-21 12:00:00-06:00,LAC,DAL,LAC,0.705734,Strong
8,2025-12-21 12:00:00-06:00,NYG,NYG,MIN,0.697276,Strong
10,2025-12-21 12:00:00-06:00,NO,NO,NYJ,0.571435,Lean
14,2025-12-21 12:00:00-06:00,TB,CAR,TB,0.517439,Coin flip
0,2025-12-21 15:05:00-06:00,ATL,ARI,ATL,0.873337,Very strong


## TEST 

In [14]:
# Identify the latest completed week
last_completed_week = (
    played["week_num"]
    .dropna()
    .astype(int)
    .max()
)

test_week = last_completed_week-1        # week we will evaluate
train_until_week = test_week - 1         # info available BEFORE predictions

test_week, train_until_week


(15, 14)

In [15]:
train_sched = played[played["week_num"] <= train_until_week].copy()
test_sched  = played[played["week_num"] == test_week].copy()

len(train_sched), len(test_sched)


(208, 16)

In [16]:
# Build long format using ONLY training games
long_home = train_sched[["game_id","week_num","home_team"]].rename(columns={"home_team":"team"})
long_away = train_sched[["game_id","week_num","away_team"]].rename(columns={"away_team":"team"})
long_train = pd.concat([long_home, long_away], ignore_index=True)

long_train = long_train.merge(feats, on=["game_id","team"], how="left")
long_train = long_train.sort_values(["week_num","game_id","team"]).copy()

WINDOW = 6

for col in ["off_ypp","def_ypp","to_margin"]:
    long_train[f"{col}_roll"] = (
        long_train.groupby("team")[col]
            .apply(lambda s: s.shift(1).rolling(WINDOW, min_periods=1).mean())
            .reset_index(level=0, drop=True)
    )

latest_train = (
    long_train
    .sort_values(["week_num","game_id"])
    .groupby("team")
    .tail(1)
    .set_index("team")[["off_ypp_roll","def_ypp_roll","to_margin_roll"]]
    .rename(columns={
        "off_ypp_roll":"off_ypp",
        "def_ypp_roll":"def_ypp",
        "to_margin_roll":"to_margin"
    })
)


/var/folders/mk/354ykzvd4xb9dv44bwrmk9w80000gn/T/ipykernel_3836/1511637698.py:14: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda s: s.shift(1).rolling(WINDOW, min_periods=1).mean())
/var/folders/mk/354ykzvd4xb9dv44bwrmk9w80000gn/T/ipykernel_3836/1511637698.py:14: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .group

In [17]:
elo_train = train_elo(train_sched, latest_train, k=20, hfa=55)


In [18]:
test_preds = []

for _, g in test_sched.iterrows():
    home, away = g["home_team"], g["away_team"]

    elo_home = elo_train.get(home, 1500.0)
    elo_away = elo_train.get(away, 1500.0)

    adj = eff_adj(home, away, latest_train)
    p_home = win_prob_from_elo(elo_home + 55 + adj, elo_away)

    test_preds.append({
        "game_id": g["game_id"],
        "home_team": home,
        "away_team": away,
        "pred_home_win_prob": p_home,
        "actual_home_win": int(g["home_score"] > g["away_score"]),
    })

test_preds = pd.DataFrame(test_preds)
test_preds


,game_id,home_team,away_team,pred_home_win_prob,actual_home_win
0,2025_15_ATL_TB,TB,ATL,0.599611,0
1,2025_15_CLE_CHI,CHI,CLE,0.370499,1
2,2025_15_BAL_CIN,CIN,BAL,0.274529,0
3,2025_15_ARI_HOU,HOU,ARI,0.738761,1
4,2025_15_NYJ_JAX,JAX,NYJ,0.897072,1
5,2025_15_LAC_KC,KC,LAC,0.824937,0
6,2025_15_BUF_NE,NE,BUF,0.583842,0
7,2025_15_WAS_NYG,NYG,WAS,0.624128,0
8,2025_15_LV_PHI,PHI,LV,0.811878,1
9,2025_15_GB_DEN,DEN,GB,0.867839,1


In [19]:
test_preds["pred_home_win"] = test_preds["pred_home_win_prob"] >= 0.5

accuracy = (
    test_preds["pred_home_win"] ==
    test_preds["actual_home_win"]
).mean()

accuracy


0.4375

In [20]:
brier = np.mean(
    (test_preds["pred_home_win_prob"] - test_preds["actual_home_win"]) ** 2
)

brier


0.2582258210757669

In [21]:
def bucket(p):
    if p < 0.55:
        return "50–55%"
    elif p < 0.65:
        return "55–65%"
    elif p < 0.75:
        return "65–75%"
    else:
        return "75%+"

test_preds["bucket"] = test_preds["pred_home_win_prob"].apply(bucket)

bucket_eval = (
    test_preds
    .groupby("bucket")
    .agg(
        games=("game_id","count"),
        actual_home_win_rate=("actual_home_win","mean")
    )
)

bucket_eval


,games,actual_home_win_rate
bucket,,
50–55%,5,0.800000
55–65%,4,0.250000
65–75%,1,1.000000
75%+,6,0.666667
